In [1]:
%config IPCompleter.greedy=True
from notebook.services.config import ConfigManager
c = ConfigManager()
c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})

{'CodeCell': {'cm_config': {'autoCloseBrackets': False}},
 'Cell': {'cm_config': {'lineNumbers': True}}}

In [2]:
from plnn.mnist_sequential import Net
import torch
import torch.nn as nn
from torchvision.datasets.mnist import MNIST
import torch.utils.data
import numpy as np
import torch.nn.functional as F
from torchvision import datasets, transforms

In [3]:
def generate_domain(input_tensor, eps_size):
    return torch.stack((input_tensor - eps_size, input_tensor + eps_size))

In [4]:
model = Net()
model.load_state_dict(torch.load('save/mnist_sequential_cnn.pt'))
model.cuda()
dataset = MNIST('./data', train=True, download=True,
                transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,), (0.3081,))
                ])),  # load the testing dataset
batch_size=10
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=False, pin_memory=True)
# test_loader = torch.utils.data.DataLoader(dataset, batch_size=1)#retrieve items 1 at a time
seed = 0

In [20]:
class VerificationNetwork(nn.Module):
    def __init__(self, base_network,true_class_index):
        super(VerificationNetwork, self).__init__()
        self.true_class_index = true_class_index
#         self.property_layers=[]
#         n_classes = base_network.layers[-1].out_features
#         print(f'n_classes={n_classes}')
#         for true_class_index in range(n_classes):
#             self.property_layers.append( self.attach_property_layers(base_network,true_class_index))
        self.property_layer = self.attach_property_layers(base_network,self.true_class_index)
        self.layers=base_network.layers+[self.property_layer]
        self.out=nn.Sequential(*self.layers)

    '''need to  repeat this method for each class so that it describes the distance between the corresponding class 
    and the closest other class'''
    def attach_property_layers(self, model: Net,true_class_index:int):
        n_classes = model.layers[-1].out_features
        cases = []
        for i in range(n_classes):
            if i == true_class_index:
                continue
            case = [0] * n_classes  # list of zeroes
            case[true_class_index] = 1  # sets the property to 1
            case[i] = -1
            cases.append(case)
        weights = np.array(cases)
#         print(f'weight={weights}')
        print(f'weights.size()={weights.shape}')
        weightTensor = nn.Linear(in_features=n_classes, out_features=n_classes-1,
                                 bias=False)
        print(f'initial weightTensor size={weightTensor.weight.size()}')
        weightTensor.weight.data = torch.from_numpy(weights).float()
        print(f'final weightTensor size={weightTensor.weight.size()}')
        return weightTensor

    def forward(self, x):
#         x = self.base_network(x)
        x = self.out(x)
        print(x)
        print(x.size())
        return torch.min(x,dim=1,keepdim=True)[0]


In [21]:
#get the data and label
data, target =next(iter(test_loader))
print(f'data size:{data.size()}')
# print(data[0])
# create the domain
domain_raw = generate_domain(data,0.001)
data_size=data.size()
print(f'domain size:{domain_raw.size()}')


data size:torch.Size([10, 1, 28, 28])
domain size:torch.Size([2, 10, 1, 28, 28])


In [22]:
print(model.layers[-1].out_features)
print(f'True class={target}')
single_true_class=7
verification_model=VerificationNetwork(model,single_true_class)
verification_model.cuda()

10
True class=tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9])
weights.size()=(9, 10)
initial weightTensor size=torch.Size([9, 10])
final weightTensor size=torch.Size([9, 10])


VerificationNetwork(
  (property_layer): Linear(in_features=10, out_features=9, bias=False)
  (out): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=9, bias=False)
  )
)

In [23]:
test_out=verification_model(data.cuda().view(-1,784))
print(f'test_out={test_out}')
# print(f'test_out[0]={test_out[0]}')

tensor([[ 11.0055,  15.3589,   8.2809,   6.8677,  14.8731,  10.9445,  21.4909,
          10.0609,   9.6480],
        [ -7.7150, -10.4823, -18.2218, -11.7193,   2.1209,  -9.6300,  -8.6503,
         -10.0324,   2.7475],
        [  5.1997,  -6.2795,  -0.4463,   1.3481,   1.1025,   1.9049,   1.9214,
          -0.2725,   3.0070],
        [ -9.4011,   2.2561,  -0.4023,   2.9498,   3.5961,   0.2800,  -0.8867,
           6.4764,   1.0052],
        [  2.4483,   4.7913,   1.2583,   4.6639,  -7.3499,   2.8749,   1.4881,
           1.9828,  -2.7216],
        [  7.9259,  -5.7664,   3.0979,   4.1033,   3.8198,   7.1183,   6.8119,
           2.6324,   5.1645],
        [  5.9190,   4.3122,   5.9192,   3.6975,  -8.7227,  -0.4677,   3.7314,
          -3.4629,  -1.3713],
        [  7.7114,   3.7077,   0.4748,  -0.4291,  -0.7820,   1.3426,   6.6008,
           2.8977,  -8.0057],
        [ -2.7740,  -3.7870,  -6.4784,  -0.0640,  -5.3700,  -9.8601,  -9.7806,
          -5.8107,  -5.3476],
        [  6.3015, 

In [24]:
test_out=model(data.cuda().view(-1,784))
print(test_out.size())

torch.Size([10, 10])


In [25]:
domain=domain_raw.view(2,batch_size,-1)
print(domain.size())

torch.Size([2, 10, 784])


In [26]:
# global_ub_point, global_ub = net.get_upper_bound(domain)
# global_lb = net.get_lower_bound(domain)


def get_upper_bound(domain,model):
    #we try get_upper_bound
    nb_samples = 1024
    nb_inp = domain.size()[2:]  #get last dimensions
    print(nb_inp)
    # Not a great way of sampling but this will be good enough
    # We want to get rows that are >= 0
    rand_samples_size = [batch_size, nb_samples] + list(nb_inp)
    print(rand_samples_size)
    rand_samples = torch.zeros(rand_samples_size)
    print(rand_samples.size())
    # print(rand_samples)
    rand_samples.uniform_(0, 1)
    # print(rand_samples)
    print(rand_samples.size())
    domain_lb = domain.select(0, 0).contiguous()
    domain_ub = domain.select(0, 1).contiguous()
    # print(domain_lb)
    print(domain_lb.size())
    # print(domain_ub)
    print(domain_ub.size())
    domain_width = domain_ub - domain_lb
    print(domain_width.size())
    print(domain_lb.view([batch_size, 1] + list(nb_inp)).size())
    print(domain_width.view([batch_size, 1] + list(nb_inp)).size())
    domain_lb = domain_lb.view([batch_size, 1] + list(nb_inp)).expand(
        [batch_size, nb_samples] + list(nb_inp))  #expand the initial point for the number of examples
    print(domain_lb.size())
    domain_width = domain_width.view([batch_size, 1] + list(nb_inp)).expand(
        [batch_size, nb_samples] + list(nb_inp))  #expand the width for the number of examples
    print(domain_width.size())
    #those should be the same
    print(domain_width.size())
    print(rand_samples.size())
    inps = domain_lb + domain_width * rand_samples
    # print(inps) #each row shuld be different
    print(inps.size())
    #now flatten the first dimension into the second
    flattened_size = [inps.size(0) * inps.size(1)] + list(inps.size()[2:])
    print(flattened_size)
    #rearrange the tensor so that is consumable by the model
    print(data_size)
    examples_data_size = [flattened_size[0]] + list(data_size[1:])  #the expected dimension of the example tensor
    print(examples_data_size)
    var_inps = torch.Tensor(inps).view(examples_data_size)
    print(f'var_inps.size()={var_inps.size()}')  #should match data_size
    print(inps.size())
    # print(inps[0][0])
    # print(inps[0][1])
    # print(var_inps[0][0])
    # print(var_inps[1][0])
    outs = model.forward(var_inps.cuda())  #gets the input for the values
    print(outs.size())
    print(outs[0])  #those two should be very similar but different because they belong to two different random examples
    print(outs[1])
    print(target.unsqueeze(1))
    target_expanded = target.unsqueeze(1).expand(
        [batch_size, nb_samples])  #generates nb_samples copies of the target vector, all rows should be the same
    print(target_expanded.size())
    print(target_expanded)
    target_idxs = target_expanded.contiguous().view(
        batch_size * nb_samples)  #contains a list of indices that tells which columns out of the 10 classes to pick
    print(target_idxs.size())  #the first dimension should match
    print(outs.size())
    print(outs[target_idxs[0]].size())
    outs_true_class = outs.gather(1, target_idxs.cuda().view(-1,
                                                             1))  #we choose dimension 1 because it's the one we want to reduce
    print(outs_true_class.size())
    # print(outs[0])
    # print(target_idxs[1])
    # print(outs[1][0])#these two should be similar but different because they belong to different examples
    # print(outs[0][0])
    print(outs_true_class.size())
    outs_true_class_resized = outs_true_class.view(batch_size, nb_samples)
    print(outs_true_class_resized.size())  #resize outputs so that they each row is a different element of each batch
    upper_bound, idx = torch.min(outs_true_class_resized,
                                 dim=1)  #this returns the distance of the network output from the given class, it selects the class which is furthest from the current one
    print(upper_bound.size())
    print(idx.size())
    print(idx)
    print(upper_bound)
    # rearranged_idx=idx.view(list(inps.size()[0:2]))
    # print(rearranged_idx.size()) #rearranged idx contains the indexes of the minimum class for each example, for each element of the batch
    print(f'idx size {idx.size()}')
    print(f'inps size {inps.size()}')
    print(idx[0])
    # upper_bound = upper_bound[0]
    unsqueezed_idx = idx.cuda().view(-1, 1)
    print(f'single size {inps[0][unsqueezed_idx[0][0]][:].size()}')
    print(f'single size {inps[1][unsqueezed_idx[1][0]][:].size()}')
    print(f'single size {inps[2][unsqueezed_idx[2][0]][:].size()}')
    ub_point = [inps[x][idx[x]][:].numpy() for x in range(idx.size()[0])]
    ub_point = torch.tensor(ub_point)
    print(
        ub_point)  #ub_point represents the input that amongst all examples returns the minimum response for the appropriate class
    print(ub_point.size())
    # print(unsqueezed_idx.size())
    # ub_point = torch.gather(inps.cuda(),1,unsqueezed_idx.cuda())#todo for some reason it doesn't want to work
    # print(ub_point.size())
    return ub_point, upper_bound


In [ ]:
#test the method
get_upper_bound(domain,verification_model)

In [35]:
def get_lower_bound(domain,model):
    '''
    input_domain: Tensor containing in each row the lower and upper bound
                  for the corresponding dimension
    '''
    #now try to do the lower bound
    import gurobipy as grb
    lower_bounds = []
    upper_bounds = []
    gurobi_vars = []
    # These three are nested lists. Each of their elements will itself be a
    # list of the neurons after a layer.

    gurobi_model = grb.Model()
    gurobi_model.setParam('OutputFlag', False)
    gurobi_model.setParam('Threads', 1)

    input_domain=domain.select(1,0)#we use a single domain, not ready for parallelisation yet
    print(input_domain.size())

    ## Do the input layer, which is a special case
    inp_lb = []
    inp_ub = []
    inp_gurobi_vars = []
    for dim in range(input_domain.size()[1]):
        ub=input_domain[1][dim]#check this value, it can be messed up
        lb=input_domain[0][dim]
    #     print(f'ub={ub} lb={lb}')
        assert ub>lb , "ub should be greater that lb"
        #     print(f'ub={ub} lb={lb}')
        v = gurobi_model.addVar(lb=lb, ub=ub, obj=0,
                              vtype=grb.GRB.CONTINUOUS,
                              name=f'inp_{dim}')
        inp_gurobi_vars.append(v)
        inp_lb.append(lb)
        inp_ub.append(ub)
    gurobi_model.update()

    lower_bounds.append(inp_lb)
    upper_bounds.append(inp_ub)
    gurobi_vars.append(inp_gurobi_vars)

    # print(lower_bounds[0][0])
    # print(upper_bounds[0][0])

    # print(model.layers[0])
    # print(range(0))

    layer_idx = 1
    for layer in model.layers:
        print(f'layer_idx={layer_idx}')
        # layer = model.layers[0]
        new_layer_lb = []
        new_layer_ub = []
        new_layer_gurobi_vars = []
        if type(layer) is nn.Linear:
            print(f'Linear')
            for neuron_idx in range(layer.weight.size(0)):
                if(layer.bias is None):
                    ub = 0
                    lb = 0
                    lin_expr = 0
                else:
                    ub = layer.bias.data[neuron_idx]
                    lb = layer.bias.data[neuron_idx]
                    lin_expr = layer.bias.data[neuron_idx].item() #adds the bias to the linear expression
            #     print(f'bias_ub={ub} bias_lb={lb}')
                
                for prev_neuron_idx in range(layer.weight.size(1)):
                    coeff = layer.weight.data[neuron_idx, prev_neuron_idx]#picks the weight between the two neurons
            #         print(f'coeff={coeff} upper={coeff*upper_bounds[-1][prev_neuron_idx]} lower={coeff*lower_bounds[-1][prev_neuron_idx]}')
    #                 assert coeff*lower_bounds[-1][prev_neuron_idx]!=coeff*upper_bounds[-1][prev_neuron_idx], f"coeff={coeff} upper={coeff*upper_bounds[-1][prev_neuron_idx]} lower={coeff*lower_bounds[-1][prev_neuron_idx]}"
                    if coeff>=0:
                        ub = ub+ coeff*upper_bounds[-1][prev_neuron_idx]#multiplies the ub
                        lb = lb+ coeff*lower_bounds[-1][prev_neuron_idx]#multiplies the lb
                    else: #inverted
                        ub = ub+ coeff*lower_bounds[-1][prev_neuron_idx]#multiplies the ub
                        lb = lb+ coeff*upper_bounds[-1][prev_neuron_idx]#multiplies the lb
            #         print(f'ub={ub} lb={lb}')
#                     assert ub!=lb
                    lin_expr = lin_expr+ coeff.item() * gurobi_vars[-1][prev_neuron_idx]#multiplies the unknown by the coefficient
            #         print(lin_expr)
                v = gurobi_model.addVar(lb=lb, ub=ub, obj=0,
                                          vtype=grb.GRB.CONTINUOUS,
                                          name=f'lay{layer_idx}_{neuron_idx}')
                gurobi_model.addConstr(v == lin_expr)
                gurobi_model.update()
            #     print(f'v={v}')
                gurobi_model.setObjective(v, grb.GRB.MINIMIZE)
                gurobi_model.optimize()
            #          print(f'gurobi status {gurobi_model.status}')
                assert gurobi_model.status == 2, "LP wasn't optimally solved"
                # We have computed a lower bound
                lb = v.X
                v.lb = lb

                # Let's now compute an upper bound
                gurobi_model.setObjective(v, grb.GRB.MAXIMIZE)
                gurobi_model.update()
                gurobi_model.reset()
                gurobi_model.optimize()
                assert gurobi_model.status == 2, "LP wasn't optimally solved"
                ub = v.X
                v.ub = ub

                new_layer_lb.append(lb)
                new_layer_ub.append(ub)
                new_layer_gurobi_vars.append(v)
        elif type(layer) == nn.ReLU:
            print('Relu')
            for neuron_idx, pre_var in enumerate(gurobi_vars[-1]):
                pre_lb = lower_bounds[-1][neuron_idx]
                pre_ub = upper_bounds[-1][neuron_idx]

                v = gurobi_model.addVar(lb=max(0, pre_lb),
                                      ub=max(0, pre_ub),
                                      obj=0,
                                      vtype=grb.GRB.CONTINUOUS,
                                      name=f'ReLU{layer_idx}_{neuron_idx}')
                if pre_lb >= 0 and pre_ub >= 0:
                    # The ReLU is always passing
                    gurobi_model.addConstr(v == pre_var)
                    lb = pre_lb
                    ub = pre_ub
                elif pre_lb <= 0 and pre_ub <= 0:
                    lb = 0
                    ub = 0
                    # No need to add an additional constraint that v==0
                    # because this will be covered by the bounds we set on
                    # the value of v.
                else:
                    lb = 0
                    ub = pre_ub
                    gurobi_model.addConstr(v >= pre_var)

                    slope = pre_ub / (pre_ub - pre_lb)
                    bias = - pre_lb * slope
                    gurobi_model.addConstr(v <= slope * pre_var + bias)

                new_layer_lb.append(lb)
                new_layer_ub.append(ub)
                new_layer_gurobi_vars.append(v)
        else:
            raise NotImplementedError
        lower_bounds.append(new_layer_lb)
        upper_bounds.append(new_layer_ub)
        gurobi_vars.append(new_layer_gurobi_vars)

        layer_idx += 1
    # Assert that this is as expected a network with a single output
    # assert len(gurobi_vars[-1]) == 1, "Network doesn't have scalar output"
    
    #last layer, minimise
    v = gurobi_model.addVar(lb=min(lower_bounds[-1]), ub=max(upper_bounds[-1]), obj=0,
                                          vtype=grb.GRB.CONTINUOUS,
                                          name=f'lay{layer_idx}_min')
#     gurobi_model.addConstr(v == min(gurobi_vars[-1]))
    gurobi_model.addGenConstrMin(v, gurobi_vars[-1], name= "minconstr")
    gurobi_model.update()
#     print(f'v={v}')
    gurobi_model.setObjective(v, grb.GRB.MINIMIZE)
    gurobi_model.optimize()
    
    gurobi_model.update()
    gurobi_vars.append([v])
   
    # We will first setup the appropriate bounds for the elements of the
    # input
    #is it just to be sure?
    for var_idx, inp_var in enumerate(gurobi_vars[0]):
        inp_var.lb = domain[0,0,var_idx]
        inp_var.ub = domain[1,0,var_idx]

    # We will make sure that the objective function is properly set up
    gurobi_model.setObjective(gurobi_vars[-1][0], grb.GRB.MINIMIZE)
    print(f'gurobi_vars[-1][0].size()={len(gurobi_vars[-1])}')
    # We will now compute the requested lower bound
    gurobi_model.update()
    gurobi_model.optimize()
    assert gurobi_model.status == 2, "LP wasn't optimally solved"
    print(f'gurobi status {gurobi_model.status}')
    print(f'Result={gurobi_vars[-1][0].X}')
    print(f'Result={gurobi_vars[-1]}')
    print(f'Result -1={gurobi_vars[-2]}')
    return gurobi_vars[-1][0].X

In [36]:
get_lower_bound(domain,verification_model)

torch.Size([2, 784])
layer_idx=1
Linear
layer_idx=2
Relu
layer_idx=3
Linear
layer_idx=4
Linear
gurobi_vars[-1][0].size()=1
gurobi status 2
Result=6.822462837675309
Result=[<gurobi.Var lay5_min (value 6.822462837675309)>]
Result -1=[<gurobi.Var lay4_0 (value 10.984329808487798)>, <gurobi.Var lay4_1 (value 15.333439391622132)>, <gurobi.Var lay4_2 (value 8.257162653656147)>, <gurobi.Var lay4_3 (value 6.822462837675309)>, <gurobi.Var lay4_4 (value 14.8620829434579)>, <gurobi.Var lay4_5 (value 10.913033766485475)>, <gurobi.Var lay4_6 (value 21.467261508123308)>, <gurobi.Var lay4_7 (value 10.029630793629961)>, <gurobi.Var lay4_8 (value 9.626551031901219)>]


6.822462837675309